# InfiniteOpt.jl: Deployment Tools
To conclude our tutorials, we'll highlight how we can use `InfiniteOpt.jl` to deploy innovative techniques for infinite-dimensional optimization to help drive/accelerate research and to promote the accessibility of advanced solution methods.

Note the upcoming `v0.6` will make for an even better and more general extension API.

## Resources
The primary place to learn more is our website: https://pulsipher.github.io/InfiniteOpt.jl/stable/develop/extensions/

We provide templates, tutorials, and manuals to get you going.

Also reach out via the forum for more help: https://github.com/pulsipher/InfiniteOpt.jl/discussions

## Modeling Objects
The principle motivation for `InfiniteOpt.jl` is to provide a flexible API to capture general InfiniteOpt problems. Accordingly, there is opportunity to add modeling objects not natively included.

### Infinite Domains
Infinite domains fall under 2 classes in `InfiniteOpt.jl`:
- Domains for scalar infinite parameters under the `InfiniteScalarDomain` abstract type
- Domains for multivariate infinite parameters under the `InfiniteArrayDomain` abstract type

Natively, we support:
- Interval domains $[lb, ub]$ (bounds can be $\pm\infty$) via `IntervalDomain`
- Univariate distribution co-domains via `UniDistributionDomain`
- Multivariate distribution co-domains via `MultiDistributionDomain`
- Combinations of scalar domains via `CollectionDomain`

Adding a new domain type is a simple as defining a new struct. Let's try it out by making a type for disjoint domains $[lb_1, ub_1] \cup [lb_2, ub_2]$:

In [ ]:
using InfiniteOpt

struct DisjointDomain <: InfiniteOpt.InfiniteScalarDomain
    lb1::Float64
    ub1::Float64
    lb2::Float64
    ub2::Float64
    # constructor
    function DisjointDomain(lb1::Real, ub1::Real, lb2::Real, ub2::Real)
        if lb1 > ub1 || lb2 > ub2 || ub1 > lb2
            error("Invalid bounds")
        end
        return new(convert(Float64, lb1), convert(Float64, ub1),
                   convert(Float64, lb2), convert(Float64, ub2))
    end
end

Now we can define an infinite parameter with this new infinite domain type:

In [ ]:
model = InfiniteModel()
@infinite_parameter(model, s ∈ DisjointDomain(0, 1, 3, 4))

Now we can represent InfiniteOpt problems with disjoint domains. To complete such an extension, we'll need to define a few functions InfiniteOpt problems with disjoint domains can be transformed. 

To the docs: https://pulsipher.github.io/InfiniteOpt.jl/stable/develop/extensions/#Infinite-Domains.

### Measure Operators
Currently, the abstraction for measure operators stores 2 things:
- The expression to be measured
- A generic data field inherited from `MeasureData` to define what the measure is.

Hence, defining new measure types is as simple as making a new `MeasureData` type.

To the docs: https://pulsipher.github.io/InfiniteOpt.jl/stable/develop/extensions/#meas_data_ext.

Note that currently, the measure API is intertwined with the transformation API. This will be decoupled and simplified in `v0.6`.

## Measure and Derivative Approximations
With the ability to represent generally represent InfiniteOpt problems, we wish to add ways to approximate them.

Without the need to write a whole new backend, we can extend a few functions to define new approximation methods. Note that this interface is currently in `InfiniteOpt.jl`, but will be transferred to `TranscriptionOpt` once it is fully decoupled from `InfiniteOpt.jl` in `v0.6`.

### Derivatives
We can extend how derivatives are approximated by simply creating 1 struct and extending 1 function, that's it!

To the docs: https://pulsipher.github.io/InfiniteOpt.jl/stable/develop/extensions/#Derivative-Evaluation-Methods.

### Measures
We can extend how measures are approximated. There are 3 ways to do this:
- Create a `DiscreteMeasureData` object (easiest, but least robust)
- For integrals, define a new evaluation method (similar to derivative approximations)
- Create a new `MeasureData` type and define how to approximate it

These are all documented in the extension documentation. Let's try the easiest one for now.

`DiscreteMeasureData` assumes a measure approximation of the form:
$$
\sum_{i \in I} \alpha_i f(\tau_i) w(\tau_i)
$$
where $\alpha_i$ are coefficients, $w(\cdot)$ is a weighting function, and $i \in I$ indexes the support points. Let's make a simple approximation that uses uniformly spaced supports $\tau_i$ with $\alpha_i = \frac{ub - lb}{|I|}$:

In [ ]:
function uniform_grid(param, num_supports)
    lb = lower_bound(param)
    ub = upper_bound(param)
    supps = collect(LinRange(lb, ub, num_supports))
    coeffs = ones(num_supports) / num_supports * (ub - lb)
    return DiscreteMeasureData(param, coeffs, supps, lower_bound = lb, upper_bound = ub)
end

Now let's try it out:

In [ ]:
model = InfiniteModel()
@infinite_parameter(model, t in [0, 5])
@variable(model, y, Infinite(t))

tdata = uniform_grid(t, 6)
y_meas = measure(y, tdata)

We can preview the approximation via `expand`:

In [ ]:
expand(y_meas)

## Optimizer Models
Finally, we discuss how to implement new transformation/solution methods via the optimizer model API.

Note that if you wish to make a solution/transformation backend that doesn't use `JuMP.jl`, the new API with `v0.6` will make this possible.

This will require some more time to write than the above extensions, but we provide a template and guide to make this straightforward: https://pulsipher.github.io/InfiniteOpt.jl/stable/develop/extensions/#extend_optimizer_model. 

If you plan to do this, please reach out first on the forum: https://github.com/pulsipher/InfiniteOpt.jl/discussions.

### Simple Example: Steady-State
For a lightweight example, consider transforming a dynamic model to a steady-state one. Let's walk through an extension to implement this.

First, we need to create an augmented `JuMP.Model` to work as our backend:

In [ ]:
mutable struct SteadyStateData
    # variable and constraint mapping
    infvar_to_svar::Dict{GeneralVariableRef, VariableRef}
    infconstr_to_sconstr::Dict{InfOptConstraintRef, ConstraintRef}
    # constructor
    function SteadyStateData()
        return new(Dict{GeneralVariableRef, VariableRef}(),
                   Dict{InfOptConstraintRef, ConstraintRef}())
    end
end

const SSKey = :SSData # Unique indenifier for the model

function SteadyStateModel(args...; kwargs...)
    # initialize the JuMP Model
    model = Model(args...; kwargs...)
    model.ext[SSKey] = SteadyStateData()
    return model
end

function steady_state_data(model::Model)
    haskey(model.ext, SSKey) || error("Model is not a SteadyStateModel.")
    return model.ext[SSKey]
end

With this, we can now define an `InfiniteModel` to use a steady-state model backend:

In [ ]:
using HiGHS

model = InfiniteModel(HiGHS.Optimizer, OptimizerModel = SteadyStateModel)

@infinite_parameter(model, t ∈ [0, 1])
@variable(model, y[1:2] >= 0, Infinite(t))
@variable(model, z)
@objective(model, Min, z + ∫(y[1] + y[2], t))
@constraint(model, c1, 2y[1] - z <= 42)
@constraint(model, c2, ∂(y[1], t) == 4y[2] - 12)

latex_formulation(model)

To apply the transform and solve the model, we'll need to extend the `build_optimizer_model!` function:

In [ ]:
## Make dispatch methods for converting InfiniteOpt expressions (assume no infinite/finite parameters, parameter functions, NLP, etc. for simplicity)
# GeneralVariableRef
function _make_expression(opt_model::Model, expr::GeneralVariableRef)
    return _make_expression(opt_model, expr, index(expr))
end
# DecisionVariableRef
function _make_expression(
    opt_model::Model, 
    expr::GeneralVariableRef, 
    ::Union{InfiniteVariableIndex, FiniteVariableIndex}
    )
    return steady_state_data(opt_model).infvar_to_svar[expr]
end
# DerivativeRef
function _make_expression(
    opt_model::Model, 
    expr::GeneralVariableRef, 
    ::DerivativeIndex
    )
    return 0.0
end
# MeasureRef --> assume is integral
function _make_expression(
    opt_model::Model, 
    expr::GeneralVariableRef,
    ::MeasureIndex
    )
    return _make_expression(opt_model, measure_function(expr))
end
# AffExpr/QuadExpr
function _make_expression(opt_model::Model, expr::Union{GenericAffExpr, GenericQuadExpr})
    return map_expression(v -> _make_expression(opt_model, v), expr)
end

# Implement the transformation (disclaimer: this makes a few simplifying assumptions)
function InfiniteOpt.build_optimizer_model!(model::InfiniteModel, key::Val{SSKey})
    # TODO check that `model` is a dynamic model
    # clear the model for a build/rebuild
    ss_model = InfiniteOpt.clear_optimizer_model_build!(model)

    # add variables
    for vref in all_variables(model)
        if index(vref) isa InfiniteVariableIndex
            start = NaN # easy hack
        else
            start = start_value(vref)
            start = isnothing(start) ? NaN : start
        end
        lb = has_lower_bound(vref) ? lower_bound(vref) : NaN
        ub = has_upper_bound(vref) ? upper_bound(vref) : NaN
        new_vref = @variable(ss_model, base_name = name(vref), lower_bound = lb, upper_bound = ub, start = start)
        steady_state_data(ss_model).infvar_to_svar[vref] = new_vref
    end

    # add the objective
    obj_func = _make_expression(ss_model, objective_function(model))
    set_objective(ss_model, objective_sense(model), obj_func)

    # add the constraints
    for cref in all_constraints(model, Union{GenericAffExpr, GenericQuadExpr})
        constr = constraint_object(cref)
        new_func = _make_expression(ss_model, constr.func)
        new_constr = build_constraint(error, new_func, constr.set)
        new_cref = add_constraint(ss_model, new_constr, name(cref))
        steady_state_data(ss_model).infconstr_to_sconstr[cref] = new_cref
    end

    # update the status --> or we get an infinite loop!
    set_optimizer_model_ready(model, true)
    return
end

Now our `InfiniteModel` can be transformed and solved. Let's transform it first and see what happens:

In [ ]:
build_optimizer_model!(model)
latex_formulation(optimizer_model(model))

It works, now let's optimize it:

In [ ]:
optimize!(model)

Now the final step is we need to use the mappings stored in `SteadyStateData` to enable solution queries. In this case, all we have to do is extend the following functions:

In [ ]:
function InfiniteOpt.optimizer_model_variable(
    vref::GeneralVariableRef,
    key::Val{SSKey}
    )
    model = optimizer_model(JuMP.owner_model(vref))
    map_dict = steady_state_data(model).infvar_to_svar
    haskey(map_dict, vref) || error("Variable $vref not used in the optimizer model.")
    return map_dict[vref]
end

function InfiniteOpt.optimizer_model_expression(
    expr::JuMP.AbstractJuMPScalar,
    key::Val{SSKey}
    )
    model = optimizer_model(InfiniteOpt._model_from_expr(expr))
    return _make_expression(model, expr)
end

function InfiniteOpt.optimizer_model_constraint(
    cref::InfOptConstraintRef,
    key::Val{SSKey}
    )
    model = optimizer_model(JuMP.owner_model(cref))
    map_dict = steady_state_data(model).infconstr_to_sconstr
    haskey(map_dict, cref) || error("Constraint $cref not used in the optimizer model.")
    return map_dict[cref]
end

Now we can make all the typical queries!

In [ ]:
@show termination_status(model)
@show primal_status(model)
@show solve_time(model)
@show objective_value(model)
@show value.(y)
@show value(z)
@show value(c1)
@show dual(c2);